# Secret Santa

### Objectives
1. Read `participants.csv` where eachand assign a secret santa assignment for each participant, given the following constraints:
    - each participant cannot be assigned themself nor significant other
    - each participant can only have one assignement and be assigned once
2. Text each participant with instructions and their secret santa assignment

### 1. make assignments

In [ ]:
import pandas as pd
import numpy as np

participants = pd.read_csv('participants.csv', index_col='id')

In [ ]:
make_assignments = True
while make_assignments:
    id_l = participants.index.tolist()
    participants['assigned_id'] = None
    participants['assigned_couple_id'] = None
    participants['assigned_name'] = None

    for index, row in participants.iterrows():
        # escape before hitting infite loop scenario
        if (len(id_l) == 1) & (id_l[0] == index):
            break
        
        needs_assignment = True

        while needs_assignment:
            rand_id = np.random.randint(low=0,high=len(id_l))
            participants.loc[index, 'assigned_id'] = id_l[rand_id]
            participants.loc[index, 'assigned_couple_id'] = participants.couple_id.loc[id_l[rand_id]]      
            participants.loc[index, 'assigned_name'] = participants.name.loc[id_l[rand_id]]

            if participants.loc[index, 'couple_id'] != participants.loc[index, 'assigned_couple_id']:
                id_l.remove(id_l[rand_id])
                needs_assignment = False
        
        # escape before hitting infite loop scenario
        if (len(id_l) == 2):
            remaining_couples_l = participants.couple_id.ix[id_l].tolist()
            if (remaining_couples_l[0] == remaining_couples_l[1]):
                break
    
    if len(participants[participants.assigned_id.isnull()]) == 0:
        make_assignments = False
        
        

participants.columns

In [ ]:
# save to csv
participants.to_csv('participants_with_assignments.csv')

### 2. text instructions and assignments

In [ ]:
import pandas as pd
import requests
import time
import datetime as dt

with open('textbelt_api.txt') as f:
    textbelt_cred = f.read().split('\n')

participants = pd.read_csv('participants_with_assignments.csv', index_col='id')
ammount = 30

In [ ]:
def build_text_message(name, assigned_name, ammount):
    msg_s = '''Hello {}, welcome to Jensen Christmas Scoopganza {}! ScoopBot 3000 here to spread Christmas cheer with Secret Santa. SCOOP!

Here's the deal:
1. I assign you someone.
2. You buy them a gift that is about ${}. But more importantly, the gift should blow their minds with your thoughtfulness.
3. On Christmas day, they get the gift. They feel extra loved.

Hold onto your butt, because your secret santa assignment is:
{}'''.format(name, dt.datetime.now().year, ammount, assigned_name)
    
    print('Characters:\t\t',len(msg_s), '\nText Messages Required:\t', len(msg_s)/153)
    return msg_s

# test
build_text_message('Bob', 'Bobette', ammount)

In [ ]:
for index, row in participants.iterrows():
    msg_s = build_text_message(row['name'], row['assigned_name'], ammount)
    
    response = requests.post('https://textbelt.com/text', {
      'phone': row['phone_number'],
      'message': msg_s,
      'key': textbelt_cred[0],
    })
        
    print(index, ' of ', len(participants))
    print(response.text)
    print('To {}; {}\n'.format(row['name'], row['phone_number']))
    
    time.sleep(4)